In [1]:
# DATASET = 'OSCE/dor_toracica_x_infarto'
DATASET = 'clicr'

# FILE = '../datasets/' + DATASET + '/respostas_ingles.txt'
FILE = '../datasets/' + DATASET + '/sequences.txt'

with open(FILE) as f:
    data_samples = f.readlines()

## Gemsim LDA

In [2]:
from gensim.test.utils import common_corpus, common_dictionary, common_texts, simple_preprocess
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel, LdaModel

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')

# def preprocess(sentences):
#     for sentence in sentences:
#         yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')

# print(data_samples[0])
data_samples2 = data_samples

# [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in data_samples2]

stop_words = stopwords.words('english')
# stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# # Define functions for stopwords, bigrams, trigrams and lemmatization
# def remove_stopwords(texts):
#     return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# data_samples2 = remove_stopwords(data_samples2)
# print(data_samples2)
# print(data_samples2)

# lista = []
# for sentence in data_samples2:
# #     print()
#     aqui = simple_preprocess(str(sentence), deacc=True)
# #     print(aqui)


# data_samples = preprocess(data_samples2)

# print('lista, :', lista)

for idx in range(len(data_samples2)):
#     print(data_samples2[idx])
    data_samples2[idx] = data_samples2[idx].lower()  # Convert to lowercase.
    data_samples2[idx] = tokenizer.tokenize(data_samples2[idx])  # Split into words.

# print(data_samples2[0])

# # Remove numbers, but not words that contain numbers.
data_samples2 = [[token for token in doc if not token.isnumeric()] for doc in data_samples2]
data_samples2 = [[token for token in doc if token not in stop_words] for doc in data_samples2]

# # Remove words that are only one character.
data_samples2 = [[token for token in doc if len(token) > 1] for doc in data_samples2]

# print(data_samples2)


dictionary = Dictionary(data_samples2)
corpus = [dictionary.doc2bow(text) for text in data_samples2]
texts = [[dictionary[word_id] for word_id, freq in doc] for doc in corpus]

# print(dictionary)
# print(corpus)
# print(data_samples2[0])



[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
# print(dictionary)
model = LdaModel(corpus, 9, dictionary)

cm = CoherenceModel(model=model, texts=texts, corpus=corpus, dictionary=dictionary, coherence='c_v')

coherence = cm.get_coherence()  # get coherence value
print('Coerência:',coherence)

# print(model.print_topics())
doc_lda = model[corpus]
# print(str(doc_lda))

# coherence_per_topic = cm.get_coherence_per_topic()
# coherence_per_topic.sort()
print('Coerência por Tópico: ',cm.get_coherence_per_topic())
# model.

Coerência: 0.7191799757808793
Coerência por Tópico:  [0.7621357147647714, 0.5744807091889002, 0.6713446163331775, 0.6420527421315805, 0.8542059649113769, 0.7205136470540203, 0.6158012489453467, 0.8424338766312275, 0.7896512620675132]


In [41]:
# Ordenar
# coherence_per_topic = cm.get_coherence_per_topic()
cm.get_coherence_per_topic().sort()
print('Coerência por Tópico: ',cm.get_coherence_per_topic())

Coerência por Tópico:  [0.7621357147647714, 0.5744807091889002, 0.6713446163331775, 0.6420527421315805, 0.8542059649113769, 0.7205136470540203, 0.6158012489453467, 0.8424338766312275, 0.7896512620675132]


In [33]:
model.show_topic(6, topn=15)

[('tumour', 0.011366457),
 ('patient', 0.00947242),
 ('case', 0.0070538786),
 ('treatment', 0.0059954505),
 ('cancer', 0.0055896738),
 ('patients', 0.005231302),
 ('tumours', 0.0049823592),
 ('ct', 0.0047540152),
 ('right', 0.0046980507),
 ('figure', 0.004651175),
 ('mass', 0.0046202606),
 ('carcinoma', 0.0044094916),
 ('left', 0.004256249),
 ('disease', 0.0041787685),
 ('breast', 0.0039463835)]

In [30]:
model.get_topic_terms(168, topn=15)

IndexError: index 168 is out of bounds for axis 0 with size 9

In [6]:
# model.top_topics()

In [7]:
import matplotlib.pyplot as plt

def plot_top_words(model, feature_names, n_top_words, title):
    fig, axes = plt.subplots(2, 5, figsize=(30, 15), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[: -n_top_words - 1 : -1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f"Topic {topic_idx +1}", fontdict={"fontsize": 30})
        ax.invert_yaxis()
        ax.tick_params(axis="both", which="major", labelsize=20)
        for i in "top right left".split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=40)

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()

## SciKit LDA

In [8]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from time import time

n_samples = 2000
n_features = 1000
n_components = 7
n_top_words = 20

# DATASET = 'OSCE/dor_toracica_x_infarto'
# FILE = '../datasets/' + DATASET + '/respostas_ingles.txt'

# with open(FILE) as f:
#     data_samples = f.readlines()

#     print(data_samples)
    
# Use tf-idf features for NMF.
print("Extracting tf-idf features for LDA...")
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.95, min_df=2, max_features=n_features, stop_words="english"
)
t0 = time()
# print(data_samples[0])

tfidf = tfidf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))


# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(
    max_df=0.95, min_df=2, max_features=n_features, stop_words="english"
)

tf = tf_vectorizer.fit_transform(data_samples)


lda = LatentDirichletAllocation(
    n_components=n_components,
    max_iter=5,
    learning_method="online",
    learning_offset=50.0,
    random_state=0,
)
t0 = time()

aux_lda = lda.fit(tfidf)

print("done in %0.3fs." % (time() - t0))

tf_feature_names = tf_vectorizer.get_feature_names_out()
plot_top_words(lda, tf_feature_names, n_top_words, "Topics in scikit LDA model")

print('LDA perplexity:', aux_lda.perplexity(tfidf[0]))
# perplexity.append(news_lda.perplexity(doc_test))

Extracting tf-idf features for LDA...


AttributeError: 'list' object has no attribute 'lower'

# ---------------------
## Exemplo tutorial

In [ ]:
# Author: Olivier Grisel <olivier.grisel@ensta.org>
#         Lars Buitinck
#         Chyi-Kwei Yau <chyikwei.yau@gmail.com>
# License: BSD 3 clause

from time import time
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

n_samples = 2000
n_features = 1000
n_components = 7
n_top_words = 20


def plot_top_words(model, feature_names, n_top_words, title):
    fig, axes = plt.subplots(2, 5, figsize=(30, 15), sharex=True)
    axes = axes.flatten()
#     print(model.components_)
    for topic_idx, topic in enumerate(model.components_):
#         print(topic)
        top_features_ind = topic.argsort()[: -n_top_words - 1 : -1]
#         print(top_features_ind)
        top_features = [feature_names[i] for i in top_features_ind]
#         print(top_features)
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f"Topic {topic_idx +1}", fontdict={"fontsize": 30})
        ax.invert_yaxis()
        ax.tick_params(axis="both", which="major", labelsize=20)
        for i in "top right left".split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=40)

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()


# Load the 20 newsgroups dataset and vectorize it. We use a few heuristics
# to filter out useless terms early on: the posts are stripped of headers,
# footers and quoted replies, and common English words, words occurring in
# only one document or in at least 95% of the documents are removed.

print("Loading dataset...")
t0 = time()
data, y = fetch_20newsgroups(
    shuffle=True,
    random_state=1,
    remove=("headers", "footers", "quotes"),
    return_X_y=True,
)
data_samples = data
# data_samples = data[:n_samples]

print("done in %0.3fs." % (time() - t0))

# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.95, min_df=2, max_features=n_features, stop_words="english"
)
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(data_samples)
# print(type(tfidf))
# print(tfidf.shape)
# print(len(tfidf[0]))

# print(tfidf.toarray())

print("done in %0.3fs." % (time() - t0))

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(
    max_df=0.95, min_df=2, max_features=n_features, stop_words="english"
)

tf = tf_vectorizer.fit_transform(data_samples)

print(
    "\n" * 2,
    "Fitting LDA models with tf features, n_samples=%d and n_features=%d..."
    % (n_samples, n_features),
)
lda = LatentDirichletAllocation(
    n_components=n_components,
    max_iter=5,
    learning_method="online",
    learning_offset=50.0,
    random_state=0,
)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

tf_feature_names = tf_vectorizer.get_feature_names_out()
plot_top_words(lda, tf_feature_names, n_top_words, "Topics in LDA model")

# ----------------

In [ ]:
from sklearn.datasets import make_multilabel_classification

# This produces a feature matrix of token counts, similar to what CountVectorizer would produce on text.
X, Y = make_multilabel_classification(random_state=0)

lda = LatentDirichletAllocation(n_components=5, random_state=0)
lda.fit(X)

# get topics for some given samples:
print(lda.transform(X[-2:]))
print('Perplexity ', lda.perplexity(X))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()

texts = ['eu voce que tal', 'ola meu bem', 'bora?']
fitted = vec.fit_transform(texts)
fitted.toarray()

In [ ]:
vec.get_feature_names_out()

In [ ]:
from gensim.test.utils import common_corpus, common_dictionary

from gensim.models.coherencemodel import CoherenceModel

topics = [

    ['human', 'computer', 'system', 'interface'],

    ['graph', 'minors', 'trees', 'eps']

]

cm = CoherenceModel(topics=topics, corpus=common_corpus, dictionary=common_dictionary, coherence='u_mass')

coherence = cm.get_coherence()  # get coherence value
print(coherence)
print(cm.get_coherence_per_topic())

In [ ]:
from gensim.corpora import Dictionary

texts2 = [['eu voce que tal', 'ola meu bem', 'bora?']]


dct = Dictionary(texts2)
bow = dct.doc2bow(["eu voce que tal", "ola meu bem"])
# bow = dct.doc2bow([])

In [ ]:
dct2 = Dictionary(["máma mele maso".split(), "ema má máma".split()])
str(dct2)

# -----------------

In [ ]:
# from gensim.models import CoherenceModel, LdaModel, HdpModel
# from gensim.corpora import Dictionary

# # texts = [['human', 'interface', 'computer'],
# #          ['survey', 'user', 'computer', 'system', 'response', 'time'],
# #          ['eps', 'user', 'interface', 'system'],
# #          ['system', 'human', 'system', 'eps'],
# #          ['user', 'response', 'time'],
# #          ['trees'],
# #          ['graph', 'trees'],
# #          ['graph', 'minors', 'trees'],
# #          ['graph', 'minors', 'survey']]
# # texts = [['human', 'interface', 'computer'], ['coronary', 'ecg']]
# texts = data_samples
# dictionary = Dictionary(texts)
# corpus = [dictionary.doc2bow(text) for text in texts]

# print(corpus)
# print(dictionary)

In [ ]:
# from tmtoolkit.topicmod.evaluate import metric_coherence_gensim
# import numpy as np
# # lda_model - LatentDirichletAllocation()
# # vect - CountVectorizer()
# # texts - the list of tokenized words
# metric_coherence_gensim(measure='c_v', 
#                         top_n=25, 
#                         topic_word_distrib=lda.components_, 
# #                         dtm=dtm_tf, 
#                         gensim_corpus  = corpus,                     
#                         vocab=np.array([x for x in vect.vocabulary_.keys()]), 
#                         texts=train['cleaned_NOUN'].values)